In [1]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np
import tensorflow as tf

from sklearn.metrics import mean_squared_error

from aepy.data import utils
from aepy.data.datasets import load_CMAPSS
from aepy.data.preprocessing import CMAPSS_preprocessor
from aepy.data.utils import evaluate
from aepy.metrics import cmapps_score
from aepy.models.base import RecurrentDecoder, RecurrentEncoder
from aepy.models.vae import VAE
from aepy.pipelines import PreprocessPipeline, TrainingPipeline

2023-12-07 00:41:45.311983: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 00:41:45.312015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 00:41:45.312751: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 00:41:45.317938: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 00:41:45.885503: W tensorflow/compiler/tf2

Using TensorFlow backend


In [2]:
# Check GPU and Tensorflow version
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

print(tf.__version__)

GPU found
2.15.0


2023-12-07 00:41:48.445150: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 00:41:48.463372: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 00:41:48.463549: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
dataset = 'FD003'
# sensors to work with: T30, T50, P30, PS30, phi
sensors = ['s_3', 's_4', 's_7', 's_11', 's_12']
# windows length
sequence_length = 30
# smoothing intensity
alpha = 0.1
# max RUL
threshold = 125

# NOT IMPLEMENTED
# x_train, y_train, x_val, y_val, x_test, y_test = utils.get_data(dataset, sensors,
# sequence_length, alpha, threshold)
train, test, y_test = load_CMAPSS('FD001')
# x_train, y_train, x_val, y_val, x_test, y_test = CMAPSS_preprocessor(train, test, y_test)
preprocess_pipeline = PreprocessPipeline(
    name='CMAPPS_preprocessing', preprocessor=CMAPSS_preprocessor)
x_train, y_train, x_val, y_val, x_test, y_test = preprocess_pipeline(
    train=train, test=test, y_test=y_test, threshold=100)

2023-12-07 00:41:51 [INFO]: +++ CMAPPS_preprocessing +++
2023-12-07 00:41:51 [INFO]: Creating folder in ../output_dir/CMAPPS_preprocessing_2023-12-07_00-41-51
2023-12-07 00:41:51 [INFO]: Selected preprocessor is a function.


In [4]:
timesteps = x_train.shape[1]
input_dim = x_train.shape[2]
intermediate_dim = 300
batch_size = 128
latent_dim = 2
epochs = 2
optimizer = 'adam'

In [5]:
train_data = dict(data=x_train, labels=y_train)
eval_data = dict(data=x_val, labels=y_val)

In [6]:
model = VAE(input_dim=(x_train.shape[1], x_train.shape[2]), latent_dim=2, 
            downstream_task='regression', encoder=RecurrentEncoder, decoder=RecurrentDecoder)
# model_callbacks = utils.get_callbacks("p", model, x_train, y_train)

2023-12-07 00:41:56 [WARNING]: No specific layer configuration has been provided. Creating default configuration...
2023-12-07 00:41:56.989447: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 00:41:56.989785: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 00:41:56.990031: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning N

In [7]:
pipeline = TrainingPipeline(model=model, num_epochs=1)
trained_model = pipeline(train_data, eval_data)  # , callbacks=model_callbacks)

2023-12-07 00:41:59 [INFO]: +++ TrainingPipeline +++
2023-12-07 00:41:59 [INFO]: Creating folder in ../output_dir/TrainingPipeline_2023-12-07_00-41-59


Cause: Unable to locate the source code of <function _gcd_import at 0x7fa36394fd80>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function _gcd_import at 0x7fa36394fd80>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @t

2023-12-07 00:42:25.803275: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904



Epoch 1: val_total_loss improved from inf to 2710.38037, saving model to ../output_dir/TrainingPipeline_2023-12-07_00-41-59/model.weights.h5
112/112 - 39s - 346ms/step - kl_loss: 121.2963 - reconstruction_loss: 0.3772 - reg_loss: 4207.9868 - total_loss: 4329.6597 - val_kl_loss: 123.6621 - val_reconstruction_loss: 0.4231 - val_reg_loss: 2586.2952 - val_total_loss: 2710.3804


In [8]:
test_data = dict(data=x_test, labels=y_test)
y_hat = trained_model.predict(test_data)

evaluate(y_true=np.expand_dims(test_data['labels'], axis=-1),
         y_hat=y_hat['reg'], sel_metric=mean_squared_error)
evaluate(y_true=np.expand_dims(test_data['labels'], axis=-1),
         y_hat=y_hat['reg'], sel_metric=cmapps_score.CMAPSS_Score())

{'z': <tf.Tensor 'vae_1/sampling_4_1/add:0' shape=(32, 2) dtype=float32>, 'z_mean': <tf.Tensor 'vae_1/encoder_1/z_mean_1/add:0' shape=(32, 2) dtype=float32>, 'z_log_var': <tf.Tensor 'vae_1/encoder_1/z_log_var_1/add:0' shape=(32, 2) dtype=float32>, 'reg': <tf.Tensor 'vae_1/regressor_1/reg_output_1/add:0' shape=(32, 1) dtype=float32>, 'recon': <tf.Tensor 'vae_1/decoder_1/lstm_2_1/transpose_1:0' shape=(32, None, 5) dtype=float32>}
3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step{'z': <tf.Tensor 'vae_1/sampling_5_1/add:0' shape=(None, 2) dtype=float32>, 'z_mean': <tf.Tensor 'vae_1/encoder_1/z_mean_1/add:0' shape=(None, 2) dtype=float32>, 'z_log_var': <tf.Tensor 'vae_1/encoder_1/z_log_var_1/add:0' shape=(None, 2) dtype=float32>, 'reg': <tf.Tensor 'vae_1/regressor_1/reg_output_1/add:0' shape=(None, 1) dtype=float32>, 'recon': <tf.Tensor 'vae_1/decoder_1/lstm_2_1/transpose_1:0' shape=(None, None, 5) dtype=float32>}
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step  
test set results: [
	 mean_squared_error: 3818.813

array(582139.03182062)